In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("data/urls.txt", "r") as f:
    urls = f.read().splitlines()

In [ ]:
for i in range(1, 418):
    page_num = "" if i == 1 else f"?page={i}"
    r = requests.get("https://www.anime-planet.com/anime/top-anime" + page_num)
    soup = BeautifulSoup(r.text, "lxml")
    for entry in soup.find_all("td", attrs={"class": "tableTitle"}):
        extract = BeautifulSoup(str(entry))
        cur_url = "https://www.anime-planet.com" + extract.find("a")["href"]
        if cur_url not in urls:
            urls.append(cur_url)
            with open("data/urls.txt", "a") as f:
                f.writeline(cur_url + "\n")

In [ ]:
urls[0], "https://www.anime-planet.com/anime/working-boys"

In [ ]:
r = requests.get(urls[0], timeout=1)
soup = BeautifulSoup(r.text)

#### title

In [ ]:
soup.find("h1").text

In [ ]:
entryBar = BeautifulSoup(str(soup.find("section", attrs={"class":"entryBar"})))

#### type, episodes, duration

In [ ]:
entryBar.find("span", attrs={"class":"type"}).text

#### studios

In [ ]:
[std.text for std in entryBar.find_all("a", {"href" : lambda l: l and l.startswith("/anime/studios/")})]

#### startYr, endYr

In [ ]:
entryBar.find("span", attrs={"class":"iconYear"}).text.strip()

In [ ]:
entryBar.find("a", {"href" : lambda l: l and l.startswith("/anime/seasons/")}).text

#### rating, votes

In [ ]:
entryBar.find("div", attrs={"class":"avgRating"})["title"]

#### description

In [ ]:
soup.find("div", attrs={"class":"pure-1 md-3-5"}).p.text

#### tags

In [ ]:
[s.text.strip() for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags"}))).find_all("a")]

#### content warning

In [ ]:
[s.text.strip(" \n,") for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags tags--plain"}))).find_all("a")]

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")

In [4]:
def getSoup(i):
    with webdriver.Chrome(options=chrome_options) as driver:
        while True:
            try:
                driver.get(urls[i])
                break
            except TimeoutException:
                pass
        
        htmlSource = driver.page_source
        
    soup = BeautifulSoup(htmlSource)
    del htmlSource
    
    return soup

In [5]:
df_list = []

In [6]:
def iterator():
    for i in range(12500, 14578, 100):
        if i == 14500:
            yield (i, 14578)
        else:
            yield (i, (i+100)) 

In [7]:
%%time

for start, stop in tqdm(iterator()):
    
    data_dict = {"title":[], "mediaTypeEpsDur":[], "studios":[], "startEnd":[], "seasonWSF":[],
                 "ratingVotes":[], "description":[], "tags":[], "contentWarn":[], 
                 "watched":[], "watching":[], "wantWatch":[], "dropped":[]}
    
    for j in range(start, stop):

        while True:
            try:
                soup = getSoup(j)
                title = soup.find("h1").text
                break
            except AttributeError:
                pass

        data_dict["title"].append(title)

        entryBar = BeautifulSoup(str(soup.find("section", attrs={"class":"entryBar"})))

        mediaTypeEpsDur = entryBar.find("span", attrs={"class":"type"})
        data_dict["mediaTypeEpsDur"].append(mediaTypeEpsDur.text if mediaTypeEpsDur else np.NaN)

        studios = [std.text for std in entryBar.find_all("a", {"href" : lambda l: l and l.startswith("/anime/studios/")})]
        data_dict["studios"].append(studios)

        startEnd = entryBar.find("span", attrs={"class":"iconYear"})
        data_dict["startEnd"].append(startEnd.text.strip() if startEnd else np.NaN)

        seasonWSF = entryBar.find("a", {"href" : lambda l: l and l.startswith("/anime/seasons/")})
        data_dict["seasonWSF"].append(seasonWSF.text if seasonWSF else np.NaN)

        ratingVotes = entryBar.find("div", attrs={"class":"avgRating"})["title"]
        data_dict["ratingVotes"].append(ratingVotes)

        description = soup.find("div", attrs={"class":"pure-1 md-3-5"}).p.text
        data_dict["description"].append(description)

        tags = [s.text.strip() for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags"}))).find_all("a")]
        data_dict["tags"].append(tags)

        contentWarn = [s.text.strip(" \n,") 
                           for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags tags--plain"}))).find_all("a")]
        data_dict["contentWarn"].append(contentWarn)

        userStats = [s.text for s in soup.find_all("span", attrs={"class":"slCount"})]
        if len(userStats) == 3:
            data_dict["watched"].append(np.NaN)
            data_dict["watching"].append(userStats[0])
            data_dict["wantWatch"].append(userStats[1])
            data_dict["dropped"].append(userStats[2])
        else:
            data_dict["watched"].append(userStats[0])
            data_dict["watching"].append(userStats[1])
            data_dict["wantWatch"].append(userStats[2])
            data_dict["dropped"].append(userStats[3])

        del soup

    df = pd.DataFrame(data_dict)
    df_list.append(df)

21it [2:12:19, 378.06s/it]

CPU times: user 1min 45s, sys: 10.6 s, total: 1min 55s
Wall time: 2h 12min 19s


In [8]:
done = pd.read_csv("data/anime.csv")

In [11]:
combined = pd.concat([done] + df_list)

In [12]:
combined.to_csv("data/anime_full.csv", index=False)

### With Multiprocessing

In [ ]:
def scraper(i):
    while True:
        try:
            soup = getSoup(i)
            title = soup.find("h1").text
            break
        except AttributeError:
            pass

    entryBar = BeautifulSoup(str(soup.find("section", attrs={"class":"entryBar"})))

    mediaTypeEpsDur = entryBar.find("span", attrs={"class":"type"})
    mediaTypeEpsDur = mediaTypeEpsDur.text if mediaTypeEpsDur else np.NaN

    studios = [std.text for std in entryBar.find_all("a", {"href" : lambda l: l and l.startswith("/anime/studios/")})]

    startEnd = entryBar.find("span", attrs={"class":"iconYear"})
    startEnd = startEnd.text.strip() if startEnd else np.NaN

    seasonWSF = entryBar.find("a", {"href" : lambda l: l and l.startswith("/anime/seasons/")})
    seasonWSF = seasonWSF.text if seasonWSF else np.NaN

    ratingVotes = entryBar.find("div", attrs={"class":"avgRating"})["title"]

    description = soup.find("div", attrs={"class":"pure-1 md-3-5"}).p.text

    tags = [s.text.strip() for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags"}))).find_all("a")]

    contentWarn = [s.text.strip(" \n,") 
                       for s in BeautifulSoup(str(soup.find("div", attrs={"class":"tags tags--plain"}))).find_all("a")]

    userStats = [s.text for s in soup.find_all("span", attrs={"class":"slCount"})]
    if len(userStats) == 3:
        watching, wantWatch, dropped = userStats
        watched = np.NaN
    else:
        watched, watching, wantWatch, dropped = userStats


    return [title, mediaTypeEpsDur, studios, startEnd, seasonWSF, ratingVotes, description, 
            tags, contentWarn, watched, watching, wantWatch, dropped]

In [ ]:
%%time
from multiprocessing import Pool
data_dict = {}

p = Pool()

data_dict["title"], data_dict["mediaTypeEpsDur"], data_dict["studios"], data_dict["startEnd"], data_dict["seasonWSF"], \
data_dict["ratingVotes"], data_dict["description"], data_dict["tags"], data_dict["contentWarn"], data_dict["watched"], \
data_dict["watching"], data_dict["wantWatch"], data_dict["dropped"] = zip(*p.map(scraper, range(64)))
    
p.close()
p.join()

df = pd.DataFrame(data_dict)